In [1]:
from azure.storage.blob import BlobServiceClient
import json

# Define the leagueId to year mapping
LEAGUE_YEAR_MAPPING = {
    "xdkosenrm3hx9eke": 2025,
    "j5a3cvfdlrql6l7a": 2024,
    "rsrawueflbxqxrbv": 2023,
    "k2mh10wll1b2c1hg": 2022,
    "x7ioxuq5kmvfgnky": 2021
}

# Connect to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=sg092620240215;AccountKey=+PaTF6WCZ0NY63Hni1XIWRJfWsnTI7QJCLVP0f1OXUoVzJyl0AcE4h2Pe1b7ZbgldGkDDFA0j9iK+AStvU4auA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
silver_client = blob_service_client.get_container_client("silver")

# Load and transform data
combined_data = {}
blob_list = silver_client.list_blobs(name_starts_with="combined_rosters_")
for blob in blob_list:
    print(f"Processing {blob.name}")
    blob_client = silver_client.get_blob_client(blob.name)
    blob_data = blob_client.download_blob().readall()
    json_data = json.loads(blob_data.decode("utf-8"))
    combined_data.update(json_data)  # Merge all files into one dict

# Add 'year' column based on leagueId
transformed_data = {}
for league_id, rosters in combined_data.items():
    year = LEAGUE_YEAR_MAPPING.get(league_id, None)  # None if leagueId not in mapping
    if year is None:
        print(f"Warning: No year mapping found for leagueId '{league_id}'")
        continue  # Skip if no year mapping exists
    
    transformed_rosters = []
    for roster in rosters:
        enriched_roster = {
            "year": year,
            **roster  # Unpack teamName, id, position, salary, status
        }
        transformed_rosters.append(enriched_roster)
    transformed_data[league_id] = transformed_rosters

# Save transformed data to silver (or a new 'gold' container)
if not transformed_data:
    raise ValueError("No data transformed")
output_blob_name = "transformed_rosters.json"  # Or include timestamp if preferred
output_blob_client = silver_client.get_blob_client(output_blob_name)
output_blob_client.upload_blob(json.dumps(transformed_data, indent=2), overwrite=True)
print(f"Uploaded {output_blob_name} to silver container")

Processing combined_rosters_20250302_000000.json
Uploaded transformed_rosters.json to silver container
